### Import libraries

In [1676]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)


In [1677]:
#Standard libraries for data analysis:
    
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm, skew, shapiro
from scipy import stats
import statsmodels.api as sm
import re #regex

# sklearn modules for data preprocessing:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#sklearn modules for Model Selection:
from sklearn import svm, tree, linear_model, neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample


#Standard libraries for data visualization:
import seaborn as sns
from scipy.stats import boxcox 
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib 
%matplotlib inline
color = sns.color_palette()
import matplotlib.ticker as mtick
from pandas.plotting import scatter_matrix
from sklearn.metrics import roc_curve



### Load dataset

In [1678]:
data = pd.read_csv('learningSet.csv')
print(data.shape)
data.head()

/var/folders/b8/vmq4cjgs7j9_w0q786_gzn500000gn/T/ipykernel_45833/2056255270.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('learningSet.csv')


(95412, 481)


,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,CLUSTER,AGE,AGEFLAG,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_5,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_10,ADATE_11,ADATE_12,ADATE_13,ADATE_14,ADATE_15,ADATE_16,ADATE_17,ADATE_18,ADATE_19,ADATE_20,ADATE_21,ADATE_22,ADATE_23,ADATE_24,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_3,RDATE_4,RDATE_5,RDATE_6,RDATE_7,RDATE_8,RDATE_9,RDATE_10,RDATE_11,RDATE_12,RDATE_13,RDATE_14,RDATE_15,RDATE_16,RDATE_17,RDATE_18,RDATE_19,RDATE_20,RDATE_21,RDATE_22,RDATE_23,RDATE_24,RAMNT_3,RAMNT_4,RAMNT_5,RAMNT_6,RAMNT_7,RAMNT_8,RAMNT_9,RAMNT_10,RAMNT_11,RAMNT_12,RAMNT_13,RAMNT_14,RAMNT_15,RAMNT_16,RAMNT_17,RAMNT_18,RAMNT_19,RAMNT_20,RAMNT_21,RAMNT_22,RAMNT_23,RAMNT_24,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,,,,XXXX,T2,36,60.0,,,,,,,NaN,NaN,F,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0,39,34,18,10,2,1,,,,5.0,,,,,,,,,,,,,,,,,,,,,X,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,9706,9606.0,9604.0,9604.

### Check for null values

In [1679]:
nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0]

,column_name,nulls_percentage
16,AGE,0.248030
23,NUMCHLD,0.870184
24,INCOME,0.223096
26,WEALTH1,0.468830
28,MBCRAFT,0.553955
...,...,...
455,RAMNT_24,0.814090
466,NEXTDATE,0.104526
467,TIMELAG,0.104526
479,CLUSTER2,0.001383


##### remove rows with more than 25% nulls

In [1680]:
# select columns
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0.25]
columns_above_threshold['column_name']

# add columns to drop a list
drop_columns_list = list(columns_above_threshold['column_name'])
print(drop_columns_list)

['NUMCHLD', 'WEALTH1', 'MBCRAFT', 'MBGARDEN', 'MBBOOKS', 'MBCOLECT', 'MAGFAML', 'MAGFEM', 'MAGMALE', 'PUBGARDN', 'PUBCULIN', 'PUBHLTH', 'PUBDOITY', 'PUBNEWFN', 'PUBPHOTO', 'PUBOPP', 'WEALTH2', 'ADATE_5', 'ADATE_10', 'ADATE_13', 'ADATE_15', 'ADATE_17', 'ADATE_19', 'ADATE_20', 'ADATE_21', 'ADATE_22', 'ADATE_23', 'ADATE_24', 'RDATE_3', 'RDATE_4', 'RDATE_5', 'RDATE_6', 'RDATE_7', 'RDATE_8', 'RDATE_9', 'RDATE_10', 'RDATE_11', 'RDATE_12', 'RDATE_13', 'RDATE_14', 'RDATE_15', 'RDATE_16', 'RDATE_17', 'RDATE_18', 'RDATE_19', 'RDATE_20', 'RDATE_21', 'RDATE_22', 'RDATE_23', 'RDATE_24', 'RAMNT_3', 'RAMNT_4', 'RAMNT_5', 'RAMNT_6', 'RAMNT_7', 'RAMNT_8', 'RAMNT_9', 'RAMNT_10', 'RAMNT_11', 'RAMNT_12', 'RAMNT_13', 'RAMNT_14', 'RAMNT_15', 'RAMNT_16', 'RAMNT_17', 'RAMNT_18', 'RAMNT_19', 'RAMNT_20', 'RAMNT_21', 'RAMNT_22', 'RAMNT_23', 'RAMNT_24']


In [1681]:
# Exclude WEALTH1 and WEALTH2 as requested 

cols = ['WEALTH1','WEALTH2']
for item in cols:
    drop_columns_list.remove(item)

#data = data.drop(columns = drop_columns_list)

There are a lot of columns that have blank spaces which represent no value in this case. They were not identified as null values by python as they are empty spaces that are read as character values by python. We will replace those values by NaNs and repeat the analysis

In [1682]:
# replace all other empty values with white space by NaN, so that we can apply the code above for dropping NaN
data = data.apply(lambda x: x.replace(" ", np.NaN))


In [1683]:
# create a dataframe to store percentage of nulls
nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0]

# re-check again which columns have more than 25% nulls
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0.25]
columns_above_threshold['column_name']

# add columns to drop a list
drop_columns_list = list(columns_above_threshold['column_name'])
print(drop_columns_list)

['MAILCODE', 'PVASTATE', 'RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP', 'AGEFLAG', 'CHILD03', 'CHILD07', 'CHILD12', 'CHILD18', 'NUMCHLD', 'WEALTH1', 'MBCRAFT', 'MBGARDEN', 'MBBOOKS', 'MBCOLECT', 'MAGFAML', 'MAGFEM', 'MAGMALE', 'PUBGARDN', 'PUBCULIN', 'PUBHLTH', 'PUBDOITY', 'PUBNEWFN', 'PUBPHOTO', 'PUBOPP', 'SOLP3', 'SOLIH', 'MAJOR', 'WEALTH2', 'GEOCODE', 'COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO', 'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS', 'WALKER', 'KIDSTUFF', 'CARDS', 'PLATES', 'LIFESRC', 'PEPSTRFL', 'ADATE_5', 'ADATE_10', 'ADATE_13', 'ADATE_15', 'ADATE_17', 'ADATE_19', 'ADATE_20', 'ADATE_21', 'ADATE_22', 'ADATE_23', 'ADATE_24', 'RFA_5', 'RFA_10', 'RFA_13', 'RFA_15', 'RFA_17', 'RFA_19', 'RFA_20', 'RFA_21', 'RFA_22', 'RFA_23', 'RFA_24', 'RDATE_3', 'RDATE_4', 'RDATE_5', 'RDATE_6', 'RDATE_7', 'RDATE_8', 'RDATE_9', 'RDATE_10', 'RDATE_11', 'RDATE_12', 'RDATE_13', 'RDATE_14', 'RDATE_15', 'RDATE_16', 'RDATE_17', 'RDATE_18', 'RDATE_19', 'RDA

In [1684]:
# Exclude WEALTH1, WEALTH2, 'VETERANS' and'SOLIH' as requested 

cols = ['MAILCODE','WEALTH1','WEALTH2','VETERANS','SOLIH']
for item in cols:
    drop_columns_list.remove(item)



In [1685]:
#drop columns in drop list
data = data.drop(columns = drop_columns_list)


In [1686]:
data.shape

(95412, 368)

### Clean categorical columns

In [1687]:
categorical = data.select_dtypes(object)
categorical.head()

,OSOURCE,STATE,ZIP,MAILCODE,NOEXCH,MDMAUD,DOMAIN,CLUSTER,HOMEOWNR,GENDER,DATASRCE,SOLIH,VETERANS,RFA_2,RFA_3,RFA_4,RFA_6,RFA_7,RFA_8,RFA_9,RFA_11,RFA_12,RFA_14,RFA_16,RFA_18,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,GRI,IL,61081,NaN,0,XXXX,T2,36,NaN,F,NaN,NaN,NaN,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,BOA,CA,91326,NaN,0,XXXX,S1,14,H,M,3,NaN,NaN,L2G,A2G,A2G,A2G,A1E,A1E,A1E,A1E,A1E,NaN,L1E,NaN,L,G,X,X,X,A
2,AMH,NC,27017,NaN,0,XXXX,R2,43,U,M,3,NaN,NaN,L4E,S4E,S4E,S4E,S4F,S4F,S4F,S4F,S4F,S4F,S4F,S4D,L,E,X,X,X,C
3,BRY,CA,95953,NaN,0,XXXX,R2,44,U,F,3,NaN,NaN,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S2D,L,E,X,X,X,C
4,NaN,FL,33176,NaN,0,XXXX,S2,16,H,F,3,12,NaN,L2F,A2F,A2F,A1D,I2D,A1E,A1E,A1E,A1E,L3D,L3D,A2D,L,F,X,X,X,A


In [1688]:
#OSOURCE & ZIP CODE

In [1689]:
drop_columns = []
drop_columns.append('OSOURCE')
drop_columns = drop_columns + ['ZIP']
drop_columns

['OSOURCE', 'ZIP']

In [1690]:
#STATE

In [1691]:
# create a dataframe to store count of donors per STATE 
df = pd.DataFrame(categorical['STATE'].value_counts()).reset_index()

# rename columns 
df.columns = ['state', 'count']

# Select only the states that have less than 2500 donors. Store information in a new variable called 'other_states' 
other_states = list(df[df['count']<2500]['state'])

# Function to rename 'other_states' 
def clean_state(x):
    if x in other_states:
        return 'other'
    else:
        return x
    
categorical['STATE'] = list(map(clean_state, categorical['STATE']))

In [1692]:
#MAILCODE

In [1693]:
categorical['MAILCODE'].value_counts(dropna = False)

NaN    94013
B       1399
Name: MAILCODE, dtype: int64

In [1694]:
#According to variables description:

# MAILCODE Mail Code
# " "= Address is OK --> Replace by A
# B = Bad Address

categorical['MAILCODE'] = categorical['MAILCODE'].fillna('A')

In [1695]:
#NOEXCH

In [1696]:
categorical['NOEXCH'].value_counts(dropna = False)

0      61203
0      33882
1        195
1         90
X         35
NaN        7
Name: NOEXCH, dtype: int64

In [1697]:
# Looking at value counts, I see that zero and one appears twice.
# This happens because one is type string and the other number. 

# According to the description:

    # NOEXCH Do Not Exchange Flag (For list rental)
    # \_ = can be exchanged
    # X = do not exchange


# Drop this column, as it is not relevant for the model



In [1698]:
drop_columns = drop_columns + ['NOEXCH']
drop_columns

['OSOURCE', 'ZIP', 'NOEXCH']

In [1699]:
# MDMAUD

In [1700]:
categorical['MDMAUD'].value_counts(dropna = False)

XXXX    95118
C1CM       65
L1CM       44
I1CM       37
D2CM       28
C2CM       24
D1CM       20
L2CM       15
L1LM        8
C1LM        8
I2CM        7
D5CM        5
D5MM        5
D2MM        4
C5CM        3
C2MM        3
C2LM        3
D5TM        3
I1LM        3
I5CM        1
C1MM        1
I5MM        1
C5MM        1
I2MM        1
L1MM        1
L2LM        1
C5TM        1
L2TM        1
Name: MDMAUD, dtype: int64

In [1701]:
# Since most of the data is unknown (XXXX) here we will drop this column

drop_columns = drop_columns + ['MDMAUD']
drop_columns

['OSOURCE', 'ZIP', 'NOEXCH', 'MDMAUD']

##### 7.02 
We will work on the column DOMAIN. Let's replace the null values with the category that is represented the most and then we will split the data into two columns DOMAIN_A and DOMAIN_B.

 - DOMAIN_A will consist of the first character from the DOMAIN
 - DOMAIN_B will consist of the second character from the DOMAIN


Then we will drop the original column DOMAIN

In [1702]:
categorical['DOMAIN'].value_counts(dropna = False)

R2     13623
T2     12369
S1     11503
S2      8530
C2      8264
C1      6145
C3      5280
T1      4982
R3      4809
U1      4510
U2      3254
U3      2598
NaN     2316
T3      2176
S3      1891
U4      1804
R1      1358
Name: DOMAIN, dtype: int64

In [1703]:
categorical['DOMAIN'] = categorical['DOMAIN'].fillna('R2')

In [1704]:
# DOMAIN_A will consist of the first character from the DOMAIN
categorical['DOMAIN_A'] = list(map(lambda x: x[0], categorical['DOMAIN']))

# DOMAIN_B will consist of the second character from the DOMAIN
categorical['DOMAIN_B'] = list(map(lambda x: x[1], categorical['DOMAIN']))
 
# Dropping old DOMAIN column
categorical.drop(columns =['DOMAIN'], inplace = True)
 
# df display
categorical

,OSOURCE,STATE,ZIP,MAILCODE,NOEXCH,MDMAUD,CLUSTER,HOMEOWNR,GENDER,DATASRCE,SOLIH,VETERANS,RFA_2,RFA_3,RFA_4,RFA_6,RFA_7,RFA_8,RFA_9,RFA_11,RFA_12,RFA_14,RFA_16,RFA_18,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2,DOMAIN_A,DOMAIN_B
0,GRI,IL,61081,A,0,XXXX,36,NaN,F,NaN,NaN,NaN,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,L,E,X,X,X,C,T,2
1,BOA,CA,91326,A,0,XXXX,14,H,M,3,NaN,NaN,L2G,A2G,A2G,A2G,A1E,A1E,A1E,A1E,A1E,NaN,L1E,NaN,L,G,X,X,X,A,S,1
2,AMH,NC,27017,A,0,XXXX,43,U,M,3,NaN,NaN,L4E,S4E,S4E,S4E,S4F,S4F,S4F,S4F,S4F,S4F,S4F,S4D,L,E,X,X,X,C,R,2
3,BRY,CA,95953,A,0,XXXX,44,U,F,3,NaN,NaN,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S2D,L,E,X,X,X,C,R,2
4,NaN,FL,33176,A,0,XXXX,16,H,F,3,12,NaN,L2F,A2F,A2F,A1D,I2D,A1E,A1E,A1E,A1E,L3D,L3D,A2D,L,F,X,X,X,A,S,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,ASE,other,99504,A,0,XXXX,27,NaN,M,NaN,NaN,NaN,L1G,F1G,F1G,F1G,P1G,P1G,NaN,NaN,NaN,NaN,NaN,NaN,L,G,X,X,X,C,C,2
95408,DCD,TX,77379,A,0,XXXX,24,H,M,3,NaN,NaN,L1F,NaN,NaN,NaN,P1F,P1F,NaN,NaN,NaN,NaN,NaN,NaN,L,F,X,X,X,A,C,1
95409,MBC,MI,48910,A,0,XXXX,30,NaN,M,NaN,NaN,NaN,L3E,S4E,S4E,S3E,NaN,A2E,N3E,N3E,N3E,N3E,F1D,F1D,L,E,X,X,X,B,C,3
95410,PRV,CA,91320,A,0,XXXX,24,H,F,2,12,NaN,L4F,S4F,A3F,S4F,S4F,S4F,S4F,S4F,S4F,S4F,S3F,S2F,L,F,X,X,X,A,C,1


In [1705]:
#CLUSTER

In [1706]:
categorical['CLUSTER'].value_counts(dropna = False)

40     3979
35     3624
36     3617
27     3553
24     3539
49     3315
12     3000
18     2897
13     2706
30     2607
39     2601
45     2515
43     2385
11     2378
51     2322
NaN    2316
14     2246
41     2176
44     1953
16     1947
2      1942
21     1891
8      1828
10     1804
46     1800
28     1630
17     1621
20     1568
53     1441
3      1428
34     1358
42     1358
23     1324
31     1285
22     1282
25     1275
38     1200
15     1173
1      1140
7      1076
5      1014
37      972
26      947
47      868
29      859
32      795
48      787
50      776
9       770
6       603
33      593
4       561
19      497
52      270
Name: CLUSTER, dtype: int64

In [1707]:
#Fill NaN with mode
categorical['CLUSTER'] = categorical['CLUSTER'].fillna('40')

In [1708]:
#HOMEOWNR

In [1709]:
categorical['HOMEOWNR'].value_counts(dropna=False)

H      52354
NaN    22228
U      20830
Name: HOMEOWNR, dtype: int64

In [1710]:
#According to variables description:

    # HOMEOWNR Home Owner Flag
    # H = Home owner
    # U = Unknown

In [1711]:
#Fill NaN with 'U'

categorical['HOMEOWNR'] = categorical['HOMEOWNR'].fillna('U')

In [1712]:
#GENDER

In [1713]:
categorical['GENDER'].value_counts(dropna=False)

F      51277
M      39094
NaN     2957
U       1715
J        365
C          2
A          2
Name: GENDER, dtype: int64

In [1714]:
#Fill NaN with the most frequent occurrence and standardize values

categorical['GENDER'] = categorical['GENDER'].fillna('F')
categorical['GENDER'] = np.where(categorical['GENDER'].isin(['U','J','C','A']),'OTHERS',categorical['GENDER'])
categorical['GENDER'].value_counts(dropna=False)

F         54234
M         39094
OTHERS     2084
Name: GENDER, dtype: int64

In [1715]:
#DATASRCE

In [1716]:
# DATASRCE Source of Overlay Data
# Indicates which third-party data source the donor
# matched against
# 1 = MetroMail
# 2 = Polk
# 3 = Both

In [1717]:
categorical['DATASRCE'].value_counts(dropna=False)

3      43549
2      23455
NaN    21280
1       7128
Name: DATASRCE, dtype: int64

In [1718]:
# Fill NaN with the most frequent occurrence 

categorical['DATASRCE'] = categorical['DATASRCE'].fillna('3')

In [1719]:
# SOLIH

In [1720]:
# SOLIH SOLICITATION LIMIT CODE IN HOUSE
# = can be mailed (Default)
# 00 = Do Not Solicit
# 01 = one solicitation per year
# 02 = two solicitations per year
# 03 = three solicitations per year
# 04 = four solicitations per year
# 05 = five solicitations per year
# 06 = six solicitations per year
# 12 = twelve solicitations per year


categorical['SOLIH'].value_counts(dropna = False)

NaN    89212
12      5693
0        296
1         94
2         75
3         19
4         16
6          7
Name: SOLIH, dtype: int64

In [1721]:
# NaN values are potencial donors. Replace by 'can be mailed'.
# Since models do not work with mix types, decided to convert numerical values into str.

categorical['SOLIH'] = categorical['SOLIH'].fillna('Can be mailed')


categorical['SOLIH'] = np.where(categorical['SOLIH'].isin(['0']),'Do Not Solicit',categorical['SOLIH'])
categorical['SOLIH'] = np.where(categorical['SOLIH'].isin(['1']),'One per year',categorical['SOLIH'])
categorical['SOLIH'] = np.where(categorical['SOLIH'].isin(['2']),'Two per year',categorical['SOLIH'])
categorical['SOLIH'] = np.where(categorical['SOLIH'].isin(['3']),'Three per year',categorical['SOLIH'])
categorical['SOLIH'] = np.where(categorical['SOLIH'].isin(['4']),'Four per year',categorical['SOLIH'])
categorical['SOLIH'] = np.where(categorical['SOLIH'].isin(['5']),'Five per year',categorical['SOLIH'])
categorical['SOLIH'] = np.where(categorical['SOLIH'].isin(['6']),'Six per year',categorical['SOLIH'])
categorical['SOLIH'] = np.where(categorical['SOLIH'].isin(['12']),'Twelve per year',categorical['SOLIH'])
    
categorical['SOLIH'].value_counts(dropna=False)


Can be mailed      89212
Twelve per year     5693
Do Not Solicit       296
One per year          94
Two per year          75
Three per year        19
Four per year         16
Six per year           7
Name: SOLIH, dtype: int64

In [1722]:
# VETERANS

In [1723]:
# VETERANS (Y/N)

categorical['VETERANS'].value_counts(dropna = False)

NaN    84986
Y      10426
Name: VETERANS, dtype: int64

In [1724]:
#Replace the NaN by 'N', since we already know the number of Y 

categorical['VETERANS'] = categorical['VETERANS'].fillna('N')

In [1725]:
# RFA codes

# An RFA (recency/frequency/monetary) field.

#                             The (current) concatenated version is a nominal
#                             or symbolic field. The individual bytes could separately be
#                             used as fields and refer to the following:

#                             First byte: Recency of Giving
#                               C=Current Donor
#                               L=Lapsed Donor
#                               I=Inactive Donor
#                               D=Dormant Donor

#                             2nd byte: Frequency of Giving
#                               1=One gift in the period of recency
#                               2=Two-Four gifts in the period of recency
#                               5=Five+ gifts in the period of recency

#                             3rd byte: Amount of Giving
#                               L=Less than $100(Low Dollar)
#                               C=$100-499(Core)
#                               M=$500-999(Major)
#                               T=$1,000+(Top)

#                             4th byte: Blank/meaningless/filler


In [1726]:
# We will keep the column RFA_2R and RFA_2A.
#These columns provide information about the status of the donor and the amount donated
# The rest of RFA columns are redundant, so I will delete them

for col_name in categorical.columns:
    if "RFA" in col_name:
        drop_columns.append(col_name)  

In [1727]:
drop_columns.remove('RFA_2R')
drop_columns.remove('RFA_2A')


In [1728]:
# MDMAUD_R and MDMAUD_A

# MDMAUD_R Recency code for MDMAUD  
# MDMAUD_F Frequecy code for MDMAUD  
# MDMAUD_A Donation Amount code for MDMAUD

# these columns are related to MDMAUD. Since most of the info is unknown I will drop them


drop_columns = drop_columns + ['MDMAUD_R']
drop_columns = drop_columns + ['MDMAUD_A']
drop_columns = drop_columns + ['MDMAUD_F']
drop_columns

['OSOURCE',
 'ZIP',
 'NOEXCH',
 'MDMAUD',
 'RFA_2',
 'RFA_3',
 'RFA_4',
 'RFA_6',
 'RFA_7',
 'RFA_8',
 'RFA_9',
 'RFA_11',
 'RFA_12',
 'RFA_14',
 'RFA_16',
 'RFA_18',
 'MDMAUD_R',
 'MDMAUD_A',
 'MDMAUD_F']

In [1729]:
# GEOCODE2

In [1730]:
# GEOCODE2 County Size Code

categorical['GEOCODE2'].value_counts(dropna=False)

A      34484
B      28505
D      16580
C      15524
NaN      319
Name: GEOCODE2, dtype: int64

In [1731]:
# Fill NaN with the most frequent occurrence 
categorical['GEOCODE2'] = categorical['GEOCODE2'].fillna('A')

##### Drop all columns from drop list

In [1732]:
categorical = categorical.drop(drop_columns, axis = 1)

In [1733]:
# check if all NaN were removed for categorical variables
nulls_percent = pd.DataFrame(categorical.isna().sum()/len(categorical)).reset_index()
nulls_percent.columns = ['column_name', 'nulls_percentage']
nulls_percent[nulls_percent['nulls_percentage']!=0]

,column_name,nulls_percentage


### Clean numerical columns

In [1734]:
# create a new dataframe with only numerical data
numerical = data.select_dtypes(np.number)
print(numerical.shape)
numerical.head()

(95412, 337)


,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_11,ADATE_12,ADATE_14,ADATE_16,ADATE_18,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2F,CLUSTER2
0,8901,0,3712,60.0,NaN,NaN,0,0,39,34,18,10,2,1,5.0,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,9706,9606.0,9604.0,9603.0,9602.0,9601.0,9511.0,9510.0,9508.0,9506.0,9503.0,9501.0,27,9702,74,6,14,240.0,31,14,5.0,9208,12.0,9402,10.0,9512,8911,9003.0,4.0,7.741935,95515,0,0.0,0,4,39.0
1,9401,1,5202,46.0,6.0,9.0,16,0,15,55,11,6,2,1,9.0,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,9706,9606.0,9604.0,9603.0,9602.0,9601.0,9511.0,9510.0,9509.0,NaN,9503.0,NaN,12,9702,32,6,13,47.0,3,1,10.0,9310,25.0,9512,25.0,9512,9310,9504.0,18.0,15.666667,148535,0,0.0,0,2,1.0
2,9001,1,0,NaN,3.0,1.0,2,0,20,29,33,6,8,1,1.0,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23

In [1735]:
# A lot of numerical columns!! Check for NaN

nulls_percent = pd.DataFrame(numerical.isna().sum()/len(numerical)).reset_index()
nulls_percent.columns = ['column_name', 'nulls_percentage']
nulls_percent_NaN = nulls_percent[nulls_percent['nulls_percentage']!=0]
nulls_percent_NaN

,column_name,nulls_percentage
3,AGE,0.248030
4,INCOME,0.223096
5,WEALTH1,0.468830
14,WEALTH2,0.459303
135,MSA,0.001383
136,ADI,0.001383
137,DMA,0.001383
302,ADATE_3,0.020438
303,ADATE_4,0.022964
304,ADATE_6,0.037280


In [1736]:
# ADATE_ columns


In [1737]:
drop_columns2 = []
for col_name in numerical.columns:
    if "ADATE" in col_name:
        drop_columns2.append(col_name)   

In [1738]:
# NEXTDATE - Date of second gift  
#Drop column. TIMELAG already takes into account whether there was a second gift or not 

drop_columns2 = drop_columns2 + ['NEXTDATE']



In [1739]:
numerical = numerical.drop(drop_columns2, axis = 1)
numerical

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2F,CLUSTER2
0,8901,0,3712,60.0,NaN,NaN,0,0,39,34,18,10,2,1,5.0,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,9208,12.0,9402,10.0,9512,8911,4.0,7.741935,95515,0,0.0,0,4,39.0
1,9401,1,5202,46.0,6.0,9.0,16,0,15,55,11,6,2,1,9.0,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,9310,25.0,9512,25.0,9512,9310,18.0,15.666667,148535,0,0.0,0,2,1.0
2,9001,1,0,NaN,3.0,1.0,2,0,20,29,33,6,8,1,1.0,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78,22,93,7,18,36,76,65,30,86,14,7,2,5,11,17,3,17,60,18,0,1,0,0,1,6,18,50,0,4,36,49,51,14,5,4,2,24,11,2,3,6,0,2,9,44,0.0,281.0,518.0,251,292,292,340,11576,32,18,20,15,12,2,0,0,1,20,1

In [1740]:
# use a predictive model to fill NaN

In [1741]:
#AGE

In [1742]:


# 1. X/y split
# 2. Remove all NaN. create a subset of the numerical data without empty spaces

numerical2 = numerical[numerical['AGE'].notna()]
X2 = numerical2.drop(['AGE','TARGET_B','TARGET_D'],axis=1)
X2 = X2.dropna(axis=1)
y2 = numerical2['AGE']


# 3. Scale data

X_normalized_2 = pd.DataFrame(MinMaxScaler().fit_transform(X2), columns=X2.columns)


# 4. Build the model

KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(X_normalized_2,y2)
KNN.score(X_normalized_2,y2)


# 5. Make predictions


X_all = numerical.drop(['AGE','TARGET_B','TARGET_D'],axis=1)
X_all = X_all.dropna(axis=1)
y_all = numerical['AGE']

X_normalized_all = pd.DataFrame(MinMaxScaler().fit_transform(X_all), columns=X_all.columns)

PREDICTIONS = pd.Series(KNN.predict(X_normalized_all),name='PREDICTIONS')


# 6. Check if both dataframes have the same lenght

print('predicted: ',len(PREDICTIONS))
print('original dataset: ',len(numerical))


# 7. Add predictions to original table

numerical = pd.concat([numerical,PREDICTIONS],axis=1)


# 8. Fill NaN

numerical['AGE'] =  np.where(numerical['AGE'].isna(), numerical['PREDICTIONS'], numerical['AGE'])


# 9. Drop calculated column with predictions
numerical = numerical.drop(['PREDICTIONS'],axis=1)


predicted:  95412
original dataset:  95412


In [1743]:
# INCOME

In [1744]:


# 1. X/y split
# 2. Remove all NaN. create a subset of the numerical data without empty spaces

numerical2 = numerical[numerical['INCOME'].notna()]
X2 = numerical2.drop(['INCOME','TARGET_B','TARGET_D'],axis=1)
X2 = X2.dropna(axis=1)
y2 = numerical2['INCOME']


# 3. Scale data

X_normalized_2 = pd.DataFrame(MinMaxScaler().fit_transform(X2), columns=X2.columns)


# 4. Build the model

KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(X_normalized_2,y2)
KNN.score(X_normalized_2,y2)


# 5. Make predictions


X_all = numerical.drop(['INCOME','TARGET_B','TARGET_D'],axis=1)
X_all = X_all.dropna(axis=1)
y_all = numerical['INCOME']

X_normalized_all = pd.DataFrame(MinMaxScaler().fit_transform(X_all), columns=X_all.columns)

PREDICTIONS = pd.Series(KNN.predict(X_normalized_all),name='PREDICTIONS')


# 6. Check if both dataframes have the same lenght

print('predicted: ',len(PREDICTIONS))
print('original dataset: ',len(numerical))


# 7. Add predictions to original table

numerical = pd.concat([numerical,PREDICTIONS],axis=1)


# 8. Fill NaN

numerical['INCOME'] =  np.where(numerical['INCOME'].isna(), numerical['PREDICTIONS'], numerical['INCOME'])


# 9. Drop calculated column with predictions
numerical = numerical.drop(['PREDICTIONS'],axis=1)

predicted:  95412
original dataset:  95412


In [1745]:
#WEALTH1

In [1746]:


# 1. X/y split
# 2. Remove all NaN. create a subset of the numerical data without empty spaces

numerical2 = numerical[numerical['WEALTH1'].notna()]
X2 = numerical2.drop(['WEALTH1','TARGET_B','TARGET_D'],axis=1)
X2 = X2.dropna(axis=1)
y2 = numerical2['WEALTH1']


# 3. Scale data

X_normalized_2 = pd.DataFrame(MinMaxScaler().fit_transform(X2), columns=X2.columns)


# 4. Build the model

KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(X_normalized_2,y2)
KNN.score(X_normalized_2,y2)


# 5. Make predictions


X_all = numerical.drop(['WEALTH1','TARGET_B','TARGET_D'],axis=1)
X_all = X_all.dropna(axis=1)
y_all = numerical['WEALTH1']

X_normalized_all = pd.DataFrame(MinMaxScaler().fit_transform(X_all), columns=X_all.columns)

PREDICTIONS = pd.Series(KNN.predict(X_normalized_all),name='PREDICTIONS')


# 6. Check if both dataframes have the same lenght

print('predicted: ',len(PREDICTIONS))
print('original dataset: ',len(numerical))


# 7. Add predictions to original table

numerical = pd.concat([numerical,PREDICTIONS],axis=1)


# 8. Fill NaN

numerical['WEALTH1'] =  np.where(numerical['WEALTH1'].isna(), numerical['PREDICTIONS'], numerical['WEALTH1'])


# 9. Drop calculated column with predictions
numerical = numerical.drop(['PREDICTIONS'],axis=1)

predicted:  95412
original dataset:  95412


In [1747]:
# TIMELAG

In [1748]:
numerical['TIMELAG'] = numerical['TIMELAG'].fillna(numerical['TIMELAG'].median())


In [1749]:
# WEALTH2

In [1750]:


# 1. X/y split
# 2. Remove all NaN. create a subset of the numerical data without empty spaces

numerical2 = numerical[numerical['WEALTH2'].notna()]
X2 = numerical2.drop(['WEALTH2','TARGET_B','TARGET_D'],axis=1)
X2 = X2.dropna(axis=1)
y2 = numerical2['WEALTH2']


# 3. Scale data

X_normalized_2 = pd.DataFrame(MinMaxScaler().fit_transform(X2), columns=X2.columns)


# 4. Build the model

KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(X_normalized_2,y2)
KNN.score(X_normalized_2,y2)


# 5. Make predictions


X_all = numerical.drop(['WEALTH2','TARGET_B','TARGET_D'],axis=1)
X_all = X_all.dropna(axis=1)
y_all = numerical['WEALTH2']

X_normalized_all = pd.DataFrame(MinMaxScaler().fit_transform(X_all), columns=X_all.columns)

PREDICTIONS = pd.Series(KNN.predict(X_normalized_all),name='PREDICTIONS')


# 6. Check if both dataframes have the same lenght

print('predicted: ',len(PREDICTIONS))
print('original dataset: ',len(numerical))


# 7. Add predictions to original table

numerical = pd.concat([numerical,PREDICTIONS],axis=1)


# 8. Fill NaN

numerical['WEALTH2'] =  np.where(numerical['WEALTH2'].isna(), numerical['PREDICTIONS'], numerical['WEALTH2'])


# 9. Drop calculated column with predictions
numerical = numerical.drop(['PREDICTIONS'],axis=1)

predicted:  95412
original dataset:  95412


In [1751]:
# MSA

In [1752]:
numerical['MSA'] = numerical['MSA'].fillna(numerical['MSA'].median())

In [1753]:
# ADI

In [1754]:
numerical['ADI'] = numerical['ADI'].fillna(numerical['ADI'].median())

In [1755]:
# DMA

In [1756]:
numerical['DMA'] = numerical['DMA'].fillna(numerical['DMA'].median())

In [1757]:
# CLUSTER2

In [1758]:
numerical['CLUSTER2'] = numerical['CLUSTER2'].fillna(np.ceil(np.mean(numerical['CLUSTER2'])))

In [1759]:
# check if all NaN were removed for numerical variables
nulls_percent = pd.DataFrame(numerical.isna().sum()/len(numerical)).reset_index()
nulls_percent.columns = ['column_name', 'nulls_percentage']
nulls_percent[nulls_percent['nulls_percentage']!=0]

,column_name,nulls_percentage


### Handling Data Imbalance in Classification Models¶

In [1760]:
# Put back together numerical and categorical data.
data = pd.concat([numerical,categorical], axis = 1)


In [1761]:
#Split the data into a training set and a test set.

X = data.drop(['TARGET_B','TARGET_D'], axis = 1)
y = data['TARGET_B']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 20)

In [1762]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(71559, 334)
(23853, 334)
(71559,)
(23853,)


In [1763]:
# Create a copy of the dataframe for the data wrangling.¶

numericals_train = X_train.select_dtypes(np.number)
numericals_test = X_test.select_dtypes(np.number)

categoricals_train= X_train.select_dtypes(include = object)
categoricals_test= X_test.select_dtypes(include = object)

In [1764]:
# Scale numerical variables with MinMaxScaler


transformer = MinMaxScaler().fit(numericals_train)

numericals_train_standardized = transformer.transform(numericals_train)
numericals_train_standardized = pd.DataFrame(numericals_train_standardized)

numericals_test_standardized = transformer.transform(numericals_test)
numericals_test_standardized = pd.DataFrame(numericals_test_standardized)


In [1765]:
#Encode the categorical variables

encoder = OneHotEncoder(handle_unknown='error', drop='first').fit(categoricals_train)


categoricals_train = encoder.transform(categoricals_train).toarray()
categoricals_train_encoded = pd.DataFrame(categoricals_train)

categoricals_test = encoder.transform(categoricals_test).toarray()
categoricals_test_encoded = pd.DataFrame(categoricals_test)

In [1766]:
# Combine both datasets

X_train_processed = pd.concat([numericals_train_standardized,categoricals_train_encoded], axis = 1)

X_test_processed = pd.concat([numericals_test_standardized,categoricals_test_encoded], axis = 1)

In [1767]:
# Fit a logistic regression model on the training data.

LR = LogisticRegression(solver='lbfgs', max_iter=600)
LR.fit(X_train_processed, y_train)

LogisticRegression(max_iter=600)

In [1768]:
predictions = LR.predict(X_test_processed)
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [1769]:
LR.score(X_test_processed, y_test)

0.9460864461493314

In [1770]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


print("precision: ",precision_score(y_test,predictions))
print("recall: ",recall_score(y_test,predictions))
print("f1: ",f1_score(y_test,predictions))


precision:  0.0
recall:  0.0
f1:  0.0


/Users/mariaesquivel/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Precision
Precision is a measure of how many of the positive predictions made are correct (true positives).

##### Recall
Recall is a measure of how many of the positive cases the classifier correctly predicted, over all the positive cases in the data.

In [1771]:
confusion_matrix(y_test,predictions)

array([[22567,     0],
       [ 1286,     0]])

Accuracy score is very high, however Precision and Recall are zero --> Accuracy paradox

This happens because data is imbalanced.

#### Downsampling (undersampling)

In [1785]:
X_train_processed.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)


In [1786]:
# only to be done on the TRAINING set
# our test set still must reflect reality!
train = pd.concat([X_train_processed, y_train], axis=1)
train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,TARGET_B
0,0.784946,0.000014,0.123687,0.876289,0.833333,1.000000,0.095436,0.000000,0.181818,0.565657,0.060606,0.020202,0.010101,0.000000,0.888889,0.015137,0.018108,0.012711,1.000000,0.0,0.000000,0.505051,0.505051,0.868687,0.010101,0.010101,0.111111,0.090909,0.000000,0.029851,0.030303,0.014925,0.065217,0.021277,0.0,0.061856,0.00,0.000000,0.023256,0.392857,0.488095,0.511905,0.369048,0.488095,0.523810,0.400000,0.353535,0.424242,0.232323,0.141414,0.171717,0.303030,0.262626,0.080808,0.030303,0.020202,0.121212,0.161616,0.353535,0.202020,0.171717,0.090909,0.010101,0.070707,0.080808,0.282828,0.646465,0.434343,0.181818,0.080808,0.676768,0.070707,0.027397,0.242424,0.410769,0.470000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.523333,0.644833,0.923077,0.846154,0.929293,0.080808,1.000000,0.010101,0.000000,0.505051,0.878788,0.808081,0.464646,0.939394,0.070707,0.040404,0.02,0.030303,0.080808,0.090909,0.050505,0.293333,0.595960,0.050505,0.000000,0.010101,0.000000,0.949495,1.000000,1.000000,1.000000,1.000000,0.525253,0.000000,0.888889,0.870588,0.831461,0.295082,0.148148,0.000000,0.000000,0.000000,0.080808,0.000000,0.000000,0.000000,0.979798,0.979798,0.979798,0.979798,0.635150,0.019969,0.911464,0.590667,0.595333,0.626667,0.633333,0.241854,0.040404,0.030303,0.050505,0.050505,0.181818,0.418182,0.18,0.147541,0.242424,0.030303,0.030303,0.050505,0.060606,0.222222,0.171717,0.20,0.090909,0.252525,0.090909,0.010101,0.474747,0.030303,0.606061,0.404040,0.131313,0.787879,0.141414,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.070707,0.000000,0.212121,0.311111,0.513158,0.262626,0.505051,0.767677,0.919192,0.606061,0.909091,0.565657,0.555556,0.414141,1.000000,1.0,0.000000,0.121212,0.262626,0.030303,0.222222,0.121212,0.000000,0.000000,0.090909,0.000000,0.101010,0.040404,0.014286,0.025316,0.010101,0.015385,0.111111,0.232323,0.020202,0.031250,0.050505,0.171717,0.101010,0.040404,0.010101,0.029851,0.060606,0.030303,0.101010,0.000000,0.020202,0.016393,0.000000,0.090909,0.858586,0.030303,0.010101,0.829412,0.010101,0.040404,0.181818,0.303030,0.297297,0.262626,0.101010,0.103093,0.252525,0.210526,0.277778,0.111111,0.000000,0.000000,0.0,0.101010,0.181818,0.033333,0.565657,0.141414,0.060606,0.070707,0.012048,0.060606,0.000000,0.054348,0.0,0.071429,0.030303,0.054545,0.014706,0.000000,0.0,0.000000,0.04,0.000000,0.00000,0.151515,0.535354,0.767677,0.090909,0.090909,0.060606,1.000000,0.888889,0.484848,0.129032,0.230769,0.030303,0.131313,0.494949,0.949495,1.000000,0.000000,0.0,0.0,0.969697,0.000000,0.040404,0.000000,0.0,0.000000,1.00000

In [1787]:
category_0 = train[train['TARGET_B'] == 0]
category_1 = train[train['TARGET_B'] == 1]


In [1788]:
category_0_undersampled = resample(category_0, 
                                   replace=False, 
                                   n_samples = len(category_1))

In [1789]:
train_undersampled = pd.concat([category_0_undersampled, category_1], axis=0)

In [1790]:
X_train_under = train_undersampled.drop('TARGET_B',axis = 1).copy()
y_train_under = train_undersampled['TARGET_B'].copy()


In [1791]:
LR_under = LogisticRegression(solver='lbfgs', max_iter=600)
LR_under.fit(X_train_under, y_train_under)
pred = LR_under.predict(X_test_processed)

print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))

precision:  0.07093391373169766
recall:  0.557542768273717
f1:  0.12585571353343863


In [1792]:
confusion_matrix(y_test,pred)

array([[13176,  9391],
       [  569,   717]])

##### Oversampling

In [1793]:
category_1_oversampled = resample(category_1, 
                                   replace=True, 
                                   n_samples = len(category_0))

In [1794]:
train_oversampled = pd.concat([category_0, category_1_oversampled], axis=0)

In [1795]:
X_train_over = train_oversampled.drop(['TARGET_B'],axis = 1).copy()
y_train_over = train_oversampled['TARGET_B'].copy()


In [1796]:
LR_over = LogisticRegression(solver='lbfgs', max_iter=1500)
LR_over.fit(X_train_over, y_train_over)
pred = LR_over.predict(X_test_processed)

print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))

precision:  0.07304515713826022
recall:  0.5295489891135303
f1:  0.12838156282401736


In [1797]:
confusion_matrix(y_test,pred)

array([[13925,  8642],
       [  605,   681]])

Both, oversampling and undersampling have improved the model.

Looking at the confusion matrix, we can see that the oversampling performed better with few false positives. 